In [4]:
import os
import cv2
import face_recognition
import numpy as np
import json
import pickle
import subprocess
import platform

# Paths
dataset_dir = "dataset"
student_data_dir = "student_data"

# Load all known faces and their details
known_faces = []
known_names = []
person_data = {}

print("🔄 Loading registered face data...")

for person_folder in os.listdir(dataset_dir):
    person_path = os.path.join(dataset_dir, person_folder)
    if not os.path.isdir(person_path):
        continue

    encoding_path = os.path.join(person_path, "encodings.pkl")
    details_path = os.path.join(person_path, "details.json")

    if os.path.exists(encoding_path) and os.path.exists(details_path):
        with open(encoding_path, "rb") as f:
            encodings = pickle.load(f)
            known_faces.extend(encodings)

        with open(details_path, "r") as f:
            details = json.load(f)

        for _ in encodings:
            known_names.append(person_folder)
            person_data[person_folder] = details
    else:
        print(f"⚠️ Missing files in {person_folder}, skipping.")

if not known_faces:
    print("❌ No known faces loaded.")
    exit()

print(f"✅ Loaded {len(known_faces)} face encodings.\n")

# Helper: Open PDF cross-platform
def open_pdf(path):
    if not os.path.exists(path):
        print(f"❌ Report not found at: {path}")
        return

    print(f"📄 Opening report...")
    try:
        if platform.system() == "Windows":
            os.startfile(path)
        elif platform.system() == "Darwin":
            subprocess.call(["open", path])
        else:  # Linux
            subprocess.call(["xdg-open", path])
    except Exception as e:
        print(f"❌ Failed to open PDF: {e}")

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera not working.")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)

    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        print("📸 Capturing image...")
        if face_locations:
            face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]
            distances = face_recognition.face_distance(known_faces, face_encoding)

            if distances.size > 0:
                best_match_index = np.argmin(distances)
                confidence = (1 - distances[best_match_index]) * 100
                matched_id = known_names[best_match_index]

                if confidence >= 60:
                    print(f"✅ Match found: {matched_id} ({confidence:.2f}%)")

                    # Open existing report from student_data/<matched_id>/Academic_Report.pdf
                    report_path = os.path.join(student_data_dir, matched_id, "Academic_Report.pdf")
                    open_pdf(report_path)
                else:
                    print(f"❌ No good match found. Confidence: {confidence:.2f}%")
            else:
                print("❌ No known faces to compare.")
        else:
            print("❌ No face detected.")

        break  # Exit after one capture

    elif key == ord('q'):
        print("🔒 Quitting.")
        break

cap.release()
cv2.destroyAllWindows()


🔄 Loading registered face data...
⚠️ Missing files in .ipynb_checkpoints, skipping.
✅ Loaded 70 face encodings.

📸 Capturing image...
✅ Match found: HAFEEZ_HASHIF_4PA22AI010 (64.68%)
📄 Opening report...
